
# Extra Tree Classifier
 Extra Trees Classifier is a type of ensemble machine learning model based on decision trees. The Extra Trees Classifier, also known as Extremely Randomized Trees, is an ensemble method that combines multiple decision trees to make predictions. It is similar to the Random Forest algorithm but adds an extra level of randomness in the tree-building process. This hyperparameter specifies the number of decision trees to include in the ensemble. In this case, we are creating an ensemble of 100 decision trees. Having more trees in the ensemble can improve the model's robustness and generalization, which can be advantageous for predictive modeling tasks. Extra Trees Classifier can provide information about the importance of different features in making predictions. This can help understand which factors are most influential in determining rental prices.


# Load necessary packages

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, make_scorer, pairwise_distances
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from utils.data_utils import *

# Loading the cleaned dataset

In [2]:
train_df = pd.read_csv('train_clean.csv')
X_test = pd.read_csv('test_clean.csv')

In [3]:
# split the train df into train and val
X = train_df.drop('monthly_rent', axis=1)
y = train_df['monthly_rent']

In [4]:
# split it into train and val datasets from the train_df
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# First look at X_train and y_train

In [5]:
X_train.head()

,rent_approval_date,flat_type,floor_area_sqm,lease_commence_date,latitude,longitude,distance_to_nearest_existing_mrt,distance_to_nearest_planned_mrt,distance_to_nearest_school,distance_to_nearest_mall,...,town_pasir ris,town_punggol,town_queenstown,town_sembawang,town_sengkang,town_serangoon,town_tampines,town_toa payoh,town_woodlands,town_yishun
49578,0.866081,0.50,0.397790,0.603774,1.308722,103.796751,0.147207,0.326400,0.507243,0.097611,...,False,False,True,False,False,False,False,False,False,False
50763,0.767289,0.75,0.563536,0.339623,1.346522,103.734843,0.201964,0.068745,0.346423,0.682251,...,False,False,False,False,False,False,False,False,False,False
24147,0.933041,0.75,0.491713,0.490566,1.367566,103.951903,0.260956,0.060712,0.028869,0.249039,...,True,False,False,False,False,False,False,False,False,False
13290,0.599341,0.25,0.215470,0.396226,1.363576,103.745977,0.336514,0.071959,0.145460,0.647535,...,False,False,False,False,False,False,False,False,False,False
17890,0.198683,0.75,0.558011,0.547170,1.346176,103.757834,0.385762,0.162398,0.062420,0.461309,...,False,False,False,False,False,False,False,False,False,False


In [6]:
y_train.head()

49578    4000
50763    4000
24147    3550
13290    2750
17890    2000
Name: monthly_rent, dtype: int64

# Using GridSearchCV to get best model

In [16]:
# Create a param grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}


# Create the KNN regressor
model = ExtraTreesClassifier(n_estimators=100, random_state=0)

# Define the scoring metric (MSE)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Create the GridSearchCV object with verbose logging
grid_search = GridSearchCV(model, param_grid, scoring=mse_scorer, cv=5, verbose=3)

# Fit the model to the data (X_train, y_train)
grid_search.fit(X_train, y_train)

# Get the best estimator (model)
best_extra_tree_regressor = grid_search.best_estimator_

# Make predictions on the validation data
y_pred = best_extra_tree_regressor.predict(X_val)


# Calculate the Root Mean Squared Error (RMSE)
rms = mean_squared_error(y_val, y_pred) ** 0.5

# Print the best hyperparameters and RMSE
print("Best Hyperparameters:", grid_search.best_params_)
print(f"Root Mean Squared Error: {rms}")


Fitting 5 folds for each of 162 candidates, totalling 810 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 1/5] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=-319277.346 total time=   2.5s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 2/5] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=-320197.299 total time=   2.8s


...

[CV 4/5] END max_depth=20, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=-352121.427 total time=   8.1s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV 5/5] END max_depth=20, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=-357360.498 total time=   8.2s
[CV 1/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=-427814.700 total time=   9.7s
[CV 2/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=-416122.058 total time=   9.8s
[CV 3/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=-405421.658 total time=   9.4s
[CV 4/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=-423927.596 total time=   9.6s
[CV 5/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=-432400.956 total time=  10.8s
[CV 1/5] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=-429557.371 total time=  19.9

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Root Mean Squared Error: 572.8914806674867


# Results obtained
Best Hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}

Root Mean Squared Error: 572.8914806674867

# Making predictions on X_test using the best extra tree regressor model

In [19]:
y_pred = best_extra_tree_regressor.predict(X_test)

save_test_predictions_in_kaggle_format(y_pred, "Extra-Tree-Regression-2", True)

,Id,Predicted
0,0,3150
1,1,3150
2,2,3150
3,3,1900
4,4,2600
...,...,...
29995,29995,3150
29996,29996,3150
29997,29997,3150
29998,29998,3150
